In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

from const import DATA_DIR

# =========================
# LOAD DATA
# =========================
chornobyl_data_path = DATA_DIR / "chornobyl" / "data"

spatial = pd.read_csv(chornobyl_data_path / "1_Spatial_dataset.csv")
plutonium = pd.read_csv(chornobyl_data_path / "2_Plutonium_isotope_measurements.csv")

spatial.columns = spatial.columns.str.strip().str.lower()
plutonium.columns = plutonium.columns.str.strip().str.lower()

spatial = spatial.rename(columns={
    "code": "id",
    "latitude": "lat",
    "longitude": "lon"
})

PU_COL = "terrestrial_density_of_soil_contamination_with_239_240pu_kbq_m-2"

plutonium = plutonium.rename(columns={
    "code": "id",
    PU_COL: "pu_activity"
})

df = pd.merge(
    spatial[["id", "lat", "lon"]],
    plutonium[["id", "pu_activity"]],
    on="id",
    how="inner"
)

df["pu_activity"] = pd.to_numeric(df["pu_activity"], errors="coerce")
df = df.dropna()

# =========================
# LOG + NOISE FILTER
# =========================
df["pu_log"] = np.log10(df["pu_activity"])
df = df[df["pu_log"] > df["pu_log"].quantile(0.15)]

# =========================
# MARKER SIZE (CONTROLLED)
# =========================
df["marker_size"] = 6 + (df["pu_log"] - df["pu_log"].min()) * 6

# =========================
# FIGURE
# =========================
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=df["lon"],
        y=df["lat"],
        mode="markers",
        marker=dict(
            size=df["marker_size"],
            color=df["pu_log"],
            colorscale="Inferno",
            opacity=0.85,
            line=dict(width=0),
            colorbar=dict(
                title="log₁₀ Pu-239/240<br>(kBq / m²)",
                x=1.05,
                len=0.7
            )
        ),
        customdata=df["pu_activity"],
        hovertemplate=(
            "Lat: %{y:.3f}<br>"
            "Lon: %{x:.3f}<br>"
            "Pu-239/240: %{customdata:.2f} kBq/m²"
            "<extra></extra>"
        ),
        showlegend=False
    )
)

# =========================
# REACTOR MARKER
# =========================
fig.add_trace(
    go.Scatter(
        x=[30.099],
        y=[51.389],
        mode="markers",
        marker=dict(
            size=14,
            symbol="diamond",
            color="#00ffff",
            line=dict(color="black", width=2)
        ),
        name="Chornobyl Reactor",
        hovertemplate="Chornobyl Nuclear Reactor<extra></extra>"
    )
)

# =========================
# ANNOTATION
# =========================
fig.add_annotation(
    x=30.15,
    y=51.45,
    text="Plutonium hotspot<br>near reactor site",
    showarrow=True,
    arrowhead=2,
    arrowcolor="#222",
    ax=60,
    ay=-40,
    font=dict(size=12, color="#222")
)

# =========================
# LAYOUT
# =========================
fig.update_layout(
    title=dict(
        text="<b>The Invisible Footprint of Chornobyl</b><br>"
             "<span style='font-size:13px;color:#666'>Persistent plutonium hotspots decades after the accident (log scale)</span>",
        x=0.02
    ),
    # width=1000,
    # height=650,
    plot_bgcolor="white",
    paper_bgcolor="white",
    margin=dict(l=60, r=90, t=90, b=80),
    font=dict(size=12, color="#222")
)

fig.update_xaxes(
    title="Longitude",
    range=[28.5, 31.5],
    showgrid=False,
    zeroline=False,
    fixedrange=True,
    color="#222"
)

fig.update_yaxes(
    title="Latitude",
    range=[49.5, 52.5],
    showgrid=False,
    zeroline=False,
    scaleanchor="x",
    scaleratio=1,
    fixedrange=True,
    color="#222"
)

# =========================
# SOURCE
# =========================
fig.add_annotation(
    text="<i>Source: UNSCEAR – Chornobyl Environmental Measurements (Pu-239/240)</i>",
    xref="paper", yref="paper",
    x=1, y=-0.15,
    showarrow=False,
    xanchor="right",
    font=dict(size=10, color="#888")
)

fig.show()


In [2]:
from const import VISUALIZATIONS_DIR

fig.write_html(VISUALIZATIONS_DIR / "the-invisible-footprint-of-chornobyl.html")